# The four steps of causal inference

## I. Model a causal problem
- Create a causal DAG for your system of interest.

## II. Identify a target estimand under the model
- Identify the causal estimand under the assumptions of the causal DAG.

## III. Estimate the causal effect based on the identified estimand
- Estimate the estimand using any kind of Stats/ML model e.g. linear regression, random forest etc.

## IV. Refute the obtain estimate
- Peform refutations on the estimate to test its robustness 


In [2]:
import numpy as np
import pandas as pd
import patsy as ps
from statsmodels.sandbox.regression.gmm import IV2SLS
import statsmodels


import dowhy
from dowhy import CausalModel
import econml


import warnings
from sklearn.exceptions import DataConversionWarning
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LassoCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LogisticRegressionCV
from econml.deepiv import DeepIVEstimator


from econml.inference import BootstrapInference
from pandas.core.common import SettingWithCopyWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings('ignore')


# Avoid printing dataconversion warnings from sklearn
# Config dict to set the logging level
import logging.config
DEFAULT_LOGGING = {
    'version': 1,
    'disable_existing_loggers': False,
    'loggers': {
        '': {
            'level': 'WARN',
        },
    }
}

logging.config.dictConfig(DEFAULT_LOGGING)
from IPython.display import Image, display



In [3]:
from pathlib import Path
import os
import sys

cwd = Path().resolve()
PARENT_DIR = os.path.dirname(cwd)
SCRIPT_DIR = os.path.join(PARENT_DIR, 'helpers')

In [4]:
sys.path.append(SCRIPT_DIR)

In [5]:
import dowhy_helpers as dwh

In [6]:
# I/O Stuff
DATA_FILENAME = "csdh_clean.csv"
DATA_FILEPATH = "/Users/callum/Uni/GitHubRepos/surviving-the-icu/datasets/drain_data/" + DATA_FILENAME
csdh = pd.read_csv(DATA_FILEPATH)

---
## 0. Naïve Estimation (no causal inference)

In [7]:
naive_est = dwh.naive_estimate(df=csdh, treatment='drain', outcome='recurrence', treatment_type='int')
print(f"Without adjusting for any confounding, the naive causal estimate is computed as {naive_est}")

Without adjusting for any confounding, the naive causal estimate is computed as -0.09356128931064231


---
## I. Model a causal problem
* Create a causal model from the data and given graph.

In [15]:
mp_model = CausalModel(data=csdh, 
                       treatment='drain', 
                       outcome='recurrence', 
                       graph='../causal_graphs/mp_dag.dot'.replace("\n", " "))

In [16]:
data_model = CausalModel(data=csdh, 
                         treatment='drain', 
                         outcome='recurrence', 
                         graph='../causal_graphs/data_dag.dot'.replace("\n", " "))

In [17]:
small_data_model = CausalModel(data=csdh,
                               treatment='drain', 
                               outcome='recurrence', 
                               graph='../causal_graphs/small_data_dag.dot'.replace("\n", " "))

---
## II. Identify a target estimand under the model

In [18]:
mp_estimand = mp_model.identify_effect(proceed_when_unidentifiable=True)
data_estimand = data_model.identify_effect(proceed_when_unidentifiable=True)
small_data_estimand = small_data_model.identify_effect(proceed_when_unidentifiable=True)

---
## III. Instrumental variable (IV; hospital) estimator

### This estimate applies to the `data_model` and `small_data_model` only; `mp_model` contains no instruments.

The instrumental variables approach attempts to estimate the effect of $T$ on $Y$ with the help of a third variable  $Z$ that is correlated with $T$ but is uncorrelated with the error term for $Y$. In other words, the instrument $Z$ is only related with $Y$ through the directed path that goes through $T$. If these conditions are satisfied, the effect of $T$ on $Y$ can be estimated using the sample analog of:

$$\frac{Cov(Y_i, Z_i)}{Cov(T_i, Z_i)}$$

The most common method for instrumental variables estimation is the two-stage least squares (2SLS). In this approach, the cause variable $T$ is first regressed on the instrument $Z$. Then, in the second stage, the outcome of interest $Y$ is regressed on the predicted value from the first-stage model. Intuitively, the effect of $T$ on $Y$ is estimated by using only the proportion of variation in $T$ due to variation in $Z$. See https://www.aeaweb.org/articles?id=10.1257/jep.15.4.69  for a detailed discussion of the method.

In [19]:
# Wald estimate method
data_iv_est = data_model.estimate_effect(data_estimand,
                                         control_value=0,
                                         treatment_value=1,
                                         method_name='iv.instrumental_variable',
                                         method_params={'iv_instrument_name': 'hospital'},
                                         test_significance=True, 
                                         confidence_intervals=True)

In [20]:
# Wald estimate method
sdata_iv_est = small_data_model.estimate_effect(small_data_estimand,
                                                control_value=0,
                                                treatment_value=1,
                                                method_name='iv.instrumental_variable',
                                                method_params={'iv_instrument_name': 'hospital'},
                                                test_significance=True, 
                                                confidence_intervals=True)

In [21]:
print(data_iv_est)

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: iv
Estimand expression:
Expectation(Derivative(recurrence, [hospital])*Derivative([drain], [hospital])
**(-1))
Estimand assumption 1, As-if-random: If U→→recurrence then ¬(U →→{hospital})
Estimand assumption 2, Exclusion: If we remove {hospital}→{drain}, then ¬({hospital}→recurrence)

## Realized estimand
Realized estimand: Wald Estimator
Realized estimand type: nonparametric-ate
Estimand expression:
                                                                              
Expectation(Derivative(recurrence, hospital))⋅Expectation(Derivative(drain, ho

        -1
spital))  
Estimand assumption 1, As-if-random: If U→→recurrence then ¬(U →→{hospital})
Estimand assumption 2, Exclusion: If we remove {hospital}→{drain}, then ¬({hospital}→recurrence)
Estimand assumption 3, treatment_effect_homogeneity: Each unit's treatment ['drain'] is affected in the same way by common caus

In [22]:
print(sdata_iv_est)

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: iv
Estimand expression:
Expectation(Derivative(recurrence, [hospital])*Derivative([drain], [hospital])
**(-1))
Estimand assumption 1, As-if-random: If U→→recurrence then ¬(U →→{hospital})
Estimand assumption 2, Exclusion: If we remove {hospital}→{drain}, then ¬({hospital}→recurrence)

## Realized estimand
Realized estimand: Wald Estimator
Realized estimand type: nonparametric-ate
Estimand expression:
                                                                              
Expectation(Derivative(recurrence, hospital))⋅Expectation(Derivative(drain, ho

        -1
spital))  
Estimand assumption 1, As-if-random: If U→→recurrence then ¬(U →→{hospital})
Estimand assumption 2, Exclusion: If we remove {hospital}→{drain}, then ¬({hospital}→recurrence)
Estimand assumption 3, treatment_effect_homogeneity: Each unit's treatment ['drain'] is affected in the same way by common caus

In [23]:
# Two-Stage Least-Squares (2SLS) Regression
# We see congruence between this and the DoWhy method
rec_vec, endog = ps.dmatrices("recurrence ~ drain", data=csdh)
exog = ps.dmatrix("hospital", data=csdh)

m = IV2SLS(rec_vec, endog, exog).fit()
m.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          IV2SLS Regression Results                           
==============================================================================
Dep. Variable:             recurrence   R-squared:                      -0.043
Model:                         IV2SLS   Adj. R-squared:                 -0.045
Method:                     Two Stage   F-statistic:                     1.067
                        Least Squares   Prob (F-statistic):              0.302
Date:                Mon, 05 Jul 2021                                         
Time:                        09:43:27                                         
No. Observations:                 745                                         
Df Residuals:                     743                                         
Df Model:                           1                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3273      0.228      1.439      0.151      -0.119       0.774
drain         -0.2811      0.272     -1.033      0.302      -0.815       0.253
==============================================================================
Omnibus:                      352.300   Durbin-Watson:                   1.923
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1397.168
Skew:                           2.296   Prob(JB):                    4.06e-304
Kurtosis:                       7.891   Cond. No.                         4.73
==============================================================================
"""

## For IV methods, only the treatment, outcome and instrument are important; the rest of the causal graph is ignored. This explains why firstly the instrumental variable method differs so greatly from other estimation techniques and why the `data_model` and `small_data_model` are perfectly congruent.

### We carry on the rest of this notebook using the `small_data_model` since it is more lightweight and hence runs faster.

In [25]:
# small data model
dwh.print_estimate_comparison(naive_est, sdata_iv_est, 'Instrumental variable')

-------------- Causal Estimates -------------- 
Naive causal estimate is -0.09356128931064231
Instrumental variable causal estimate is -0.28110039057421726
Percentage change from naive_est: 200.445%
----------------------------------------------


In [24]:
# data model
dwh.print_estimate_comparison(naive_est, data_iv_est, 'Instrumental variable')

-------------- Causal Estimates -------------- 
Naive causal estimate is -0.09356128931064231
Instrumental variable causal estimate is -0.28110039057421726
Percentage change from naive_est: 200.445%
----------------------------------------------


---
## IV. Refute the obtained estimate

1. **Add Random Common Cause:** Does the estimation method change its estimate after we add an independent random variable as a common cause to the dataset? (Hint: It should not)

In [28]:
# Robust if: estimate stays the same
iv_ran_refuter = small_data_model.refute_estimate(small_data_estimand,
                                                  sdata_iv_est,
                                                  num_simulations=100,
                                                  method_name="random_common_cause")

2. **Placebo Treatment:** What happens to the estimated causal effect when we replace the true treatment variable with an independent random variable? (Hint: the effect should go to zero)
- Note that the placebo type is 'permute' meaning the rows of the treatment variable have been randomly permuted giving the effect of a placebo treatment.

In [30]:
# Robust if: estimate goes to 0
iv_placebo_refuter = small_data_model.refute_estimate(small_data_estimand, 
                                                      sdata_iv_est,
                                                      method_name="placebo_treatment_refuter",
                                                      num_simulations=100,
                                                      placebo_type='permute')

3. **Dummy Outcome:** What happens to the estimated causal effect when we replace the true outcome variable with an independent random variable? (Hint: The effect should go to zero)

- The result shows that when using a dummy outcome, the **treatment does not lead to the outcome**. The estimated effect is hence a value that tends to zero, which matches our expectation. This shows that if we replace the outcome by randomly generated data, the **estimator correctly predicts that the influence if treatment is zero**.

In [31]:
#### NOT APPLICABLE TO IV ESTIMATION METHODS ########
# Robust if: estimate goes to 0
# iv_dummy_refuter = data_model.refute_estimate(data_estimand,
#                                               data_iv_est,
#                                               method_name="dummy_outcome_refuter")

4. **Data Subsets Validation:** Does the estimated effect change significantly when we replace the given dataset with a randomly selected subset? (Hint: It should not)

In [32]:
# Robust if: estimate stays the same
iv_subset_refuter = small_data_model.refute_estimate(small_data_estimand, 
                                                     sdata_iv_est,
                                                     method_name="data_subset_refuter",
                                                     num_simulations=100,
                                                     subset_fraction=0.75)

5. **Bootstrap Validation:** Does the estimated effect change significantly when we replace the given dataset with bootstrapped samples from the same dataset? (Hint: It should not)

In [34]:
# Robust if: estimate stays the same
iv_bootstrap_refuter = small_data_model.refute_estimate(small_data_estimand, 
                                                        sdata_iv_est,
                                                        method_name="bootstrap_refuter", 
                                                        num_simulations=100)

In [35]:
print(iv_ran_refuter)

Refute: Add a Random Common Cause
Estimated effect:-0.28110039057421726
New effect:-0.28110039057421726



In [36]:
print(iv_placebo_refuter)

Refute: Use a Placebo Treatment
Estimated effect:-0.28110039057421726
New effect:0.003687280619944968
p value:0.49



In [37]:
print(iv_subset_refuter)

Refute: Use a subset of data
Estimated effect:-0.28110039057421726
New effect:-0.297031230083872
p value:0.48



In [38]:
print(iv_bootstrap_refuter)

Refute: Bootstrap Sample Dataset
Estimated effect:-0.28110039057421726
New effect:-0.25066498481613725
p value:0.41



In [41]:
import keras
from econml.deepiv import DeepIVEstimator
dims_zx = len(small_data_model._instruments)+len(small_data_model._effect_modifiers)
dims_tx = len(small_data_model._treatment)+len(small_data_model._effect_modifiers)
treatment_model = keras.Sequential([keras.layers.Dense(128, activation='relu', input_shape=(dims_zx,)), # sum of dims of Z and X
                                    keras.layers.Dropout(0.17),
                                    keras.layers.Dense(64, activation='relu'),
                                    keras.layers.Dropout(0.17),
                                    keras.layers.Dense(32, activation='relu'),
                                    keras.layers.Dropout(0.17)])
response_model = keras.Sequential([keras.layers.Dense(128, activation='relu', input_shape=(dims_tx,)), # sum of dims of T and X
                                   keras.layers.Dropout(0.17),
                                   keras.layers.Dense(64, activation='relu'),
                                   keras.layers.Dense(32, activation='relu'),
                                   keras.layers.Dropout(0.17),
                                   keras.layers.Dense(1)])

deepiv_estimate = data_model.estimate_effect(data_estimand,
                                             method_name="iv.econml.iv.nnet.DeepIV",
                                             confidence_intervals=False,
                                             target_units='ate',
                                             method_params={"init_params":{'n_components': 10, # Number of gaussians in the mixture density networks
                                                              'm': lambda z, x: data_treatment_model(keras.layers.concatenate([z, x])), # Treatment model,
                                                              "h": lambda t, x: data_response_model(keras.layers.concatenate([t, x])), # Response model
                                                              'n_samples': 1, # Number of samples used to estimate the response
                                                              'first_stage_options': {'epochs':25},
                                                              'second_stage_options': {'epochs':25},
                                                             },
                                               "fit_params":{}})
print(deepiv_estimate)

ModuleNotFoundError: No module named 'keras'